# Data Collection
In this lab we will take a quick look at some simple data collection and preprocessing techniques.

## Data Collection

Ever thought about grabbing some cool data for your project? Sure, you could just download a ready-made dataset that fits your needs. But let's be real - sometimes, what you need is as unique as a unicorn in a field of horses.

So, what do you do when the perfect dataset is playing hard to get? You have two awesome choices:

1. **Become a Data Detective with APIs**: Think of an API as your personal data assistant. It's like saying, "Hey API, can you fetch me some data?" And voila, it gets the job done.

2. **DIY Dataset Creation**: Roll up your sleeves and create your dataset masterpiece. How? By using techniques like web scraping or crawling for example. It's like going on a treasure hunt on the internet!

### API
Use “**The Movie DB**” API to:
    1. Download data about movies.
    2. Search for movies in the “Comedy” genre released in the year 2000 or later. Retrieve the 300 most popular movies in this genre. The movies should be sorted from most popular to least popular. Hint: Sorting based on popularity can be done in the API call.
    3. For each comedy movie, download its first 5 similar movies. If a movie has fewer than 5 similar movies, the API will return as many as it can find. Your code should be flexible to work with however many movies the API returns.
    
For more information on retrieving movie data, visit the following The Movie DB API documentation pages:
        - [Movie Discover](https://developers.themoviedb.org/3/discover/movie-discover)
        - [Get Movie List](https://developers.themoviedb.org/3/genres/get-movie-list)
        - [Get Similar Movies](ttps://developers.themoviedb.org/3/movies/get-similar-movies)

#### Saving Results
   - **File Format**: For the comedy movies, save the results in `movie_ID_name.csv`. For the comedy movies similar movies, save the results in `movie_ID_sim_movie_ID.csv`.
   - **Format Specification**: For the comedy movies, each line should describe one movie in the format `movie-ID,movie-name` without any spaces after the comma and no column headers.
   - **Example**: A line in the file could look like `353486,Jumanji: Welcome to the Jungle`.
   For the comedy movies' similar movies each line in the file should describe one pair of similar movies in the format `movie-ID,similar-movie-ID`, without any spaces after comma, and no column headers.
      - **Example**: If `Jumanji: Welcome to the Jungle` which has ID, `353486` has 3 similar movies with IDs `A`, `B` and `C` respectively then the following lines should be added to `movie_ID_sim_movie_ID.csv`
            - `353486, A`
            - `353486, B`
            - `353486, C`

#### Notes
   - **Multiple API Calls**: You may need to make multiple calls to retrieve all 300 movies, possibly retrieving them page by page due to pagination.
   - **API Parameters**: Use the `primary_release_date` parameter for movies released in 2000 or later, instead of `release_date` to avoid incorrect returns.

#### Deliverables
   - **movie_ID_name.csv**: The text file that contains the output for 2.
   - **movie_ID_sim_movie_ID.csv**: The text file that contains the output for 3.


You will need an API key to get data from the TMDb. For the purposes of this lab we will simply include it in our notebook however you should **NOT** do this for anything other than personal projects saved locally. Best practice is to have the API key be inserted at runtime via a command line argument, environment variable etc.

#### How to Use TheMovieDB API
- **Create an Account**: Sign up at [https://www.themoviedb.org/account/signup](https://www.themoviedb.org/account/signup).
- **Request API Key**:
  1. Log in and go to **Settings**.
  2. Navigate to the **API** tab in the left panel.
  3. Request an API key by selecting “Developer” and accepting the terms.
  4. Fill out the form.
  5. Your API key will be available under the API tab.

##### Important Notes
- **API Documentation**: Refer to [TheMovieDB API Documentation](https://developers.themoviedb.org/3/getting-started/introduction) for guidance.
- **Rate Limiting**: The API allows 40 requests every 10 seconds. Set appropriate timeout intervals in your code.
- **Variable Results**: The API may return different results for the same request. Plan your script's run time accordingly.


In [24]:
# !pip install requests beautifulsoup4 python-csv
import requests
import csv

In [25]:
api_key = 'YOUR_API_KEY' # Replace 'YOUR_API_KEY' with your actual API key
base_url = 'https://api.themoviedb.org/3'

In [26]:
# your code here

In [27]:
# Function to get comedy movies
def get_comedy_movies():
    movies = []
    for page in range(1, 13):  # Adjust the range based on how many movies you need
        url = f"{base_url}/discover/movie?api_key={api_key}&with_genres=35&primary_release_date.gte=2000&sort_by=popularity.desc&page={page}"
        response = requests.get(url)
        movies.extend(response.json()['results'])
        if len(movies) >= 300:
            break
    return movies[:300]

# Function to get similar movies
def get_similar_movies(movie_id):
    url = f"{base_url}/movie/{movie_id}/similar?api_key={api_key}&language=en-US&page=1"
    response = requests.get(url)
    similar_movies = response.json()['results'][:5]
    return similar_movies

# Fetch and save comedy movies
comedy_movies = get_comedy_movies()
with open('movie_ID_name.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for movie in comedy_movies:
        writer.writerow([movie['id'], movie['title']])

# Fetch and save similar movies for each comedy movie
with open('movie_ID_sim_movie_ID.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for movie in comedy_movies:
        similar_movies = get_similar_movies(movie['id'])
        for sim_movie in similar_movies:
            writer.writerow([movie['id'], sim_movie['id']])

## Web Scraping

- **Target Website**: Our goal is to scrape job listings and their details from [Fake Jobs](https://realpython.github.io/fake-jobs/).
- **Initial Step**: Start by opening the website in your browser to familiarize yourself with its layout and content.
- **Understanding HTML Structure**:
  - In Chrome, to understand the page's HTML structure, go to: 
    - `Options` -> `More Tools` -> `Developer Tools`.
  - This will help in planning our scraping strategy effectively.


Here, we set up our web scraping environment by importing necessary libraries and defining the URL of the website we want to scrape.

In [28]:
import requests
from bs4 import BeautifulSoup

URL = "https://realpython.github.io/fake-jobs/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

Extract the main container that holds the job listings on the webpage.

In [29]:
# your code here
results = soup.find(id="ResultsContainer")
results

<div class="columns is-multiline" id="ResultsContainer">
<div class="column is-half">
<div class="card">
<div class="card-content">
<div class="media">
<div class="media-left">
<figure class="image is-48x48">
<img alt="Real Python Logo" src="https://files.realpython.com/media/real-python-logo-thumbnail.7f0db70c2ed2.jpg?__no_cf_polish=1"/>
</figure>
</div>
<div class="media-content">
<h2 class="title is-5">Senior Python Developer</h2>
<h3 class="subtitle is-6 company">Payne, Roberts and Davis</h3>
</div>
</div>
<div class="content">
<p class="location">
        Stewartbury, AA
      </p>
<p class="is-small has-text-grey">
<time datetime="2021-04-08">2021-04-08</time>
</p>
</div>
<footer class="card-footer">
<a class="card-footer-item" href="https://www.realpython.com" target="_blank">Learn</a>
<a class="card-footer-item" href="https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html" target="_blank">Apply</a>
</footer>
</div>
</div>
</div>
<div class="column is-half">


Retrieve all job elements/cards from the results container.

In [30]:
job_elements = results.find_all("div", class_="card-content")
job_elements

[<div class="card-content">
 <div class="media">
 <div class="media-left">
 <figure class="image is-48x48">
 <img alt="Real Python Logo" src="https://files.realpython.com/media/real-python-logo-thumbnail.7f0db70c2ed2.jpg?__no_cf_polish=1"/>
 </figure>
 </div>
 <div class="media-content">
 <h2 class="title is-5">Senior Python Developer</h2>
 <h3 class="subtitle is-6 company">Payne, Roberts and Davis</h3>
 </div>
 </div>
 <div class="content">
 <p class="location">
         Stewartbury, AA
       </p>
 <p class="is-small has-text-grey">
 <time datetime="2021-04-08">2021-04-08</time>
 </p>
 </div>
 <footer class="card-footer">
 <a class="card-footer-item" href="https://www.realpython.com" target="_blank">Learn</a>
 <a class="card-footer-item" href="https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html" target="_blank">Apply</a>
 </footer>
 </div>,
 <div class="card-content">
 <div class="media">
 <div class="media-left">
 <figure class="image is-48x48">
 <img alt="Rea

Display the job elements to understand their structure. This includes the job title, company, location, and date/time information.

In [31]:
for job_element in job_elements:
    title_element = job_element.find("h2", class_="title")
    company_element = job_element.find("h3", class_="company")
    location_element = job_element.find("p", class_="location")
    datetime_element = job_element.find("p", class_="is-small has-text-grey")
    print(title_element)
    print("\n")
    print(company_element)
    print("\n")
    print(location_element)
    print("\n")
    print(datetime_element)
    print("\n")
    print("=====================\n")

<h2 class="title is-5">Senior Python Developer</h2>


<h3 class="subtitle is-6 company">Payne, Roberts and Davis</h3>


<p class="location">
        Stewartbury, AA
      </p>


<p class="is-small has-text-grey">
<time datetime="2021-04-08">2021-04-08</time>
</p>



<h2 class="title is-5">Energy engineer</h2>


<h3 class="subtitle is-6 company">Vasquez-Davidson</h3>


<p class="location">
        Christopherville, AA
      </p>


<p class="is-small has-text-grey">
<time datetime="2021-04-08">2021-04-08</time>
</p>



<h2 class="title is-5">Legal executive</h2>


<h3 class="subtitle is-6 company">Jackson, Chambers and Levy</h3>


<p class="location">
        Port Ericaburgh, AA
      </p>


<p class="is-small has-text-grey">
<time datetime="2021-04-08">2021-04-08</time>
</p>



<h2 class="title is-5">Fitness centre manager</h2>


<h3 class="subtitle is-6 company">Savage-Bradley</h3>


<p class="location">
        East Seanview, AP
      </p>


<p class="is-small has-text-grey">
<time da

Clean the extracted data and then use this function to print the cleaned job details.

In [32]:
def extract_and_strip_text(element):
    return element.text.strip()

for job_element in job_elements:
    title_element = job_element.find("h2", class_="title")
    company_element = job_element.find("h3", class_="company")
    location_element = job_element.find("p", class_="location")
    datetime_element = job_element.find("p", class_="is-small has-text-grey")
    print(title_element.text)
    print(extract_and_strip_text(company_element))
    print(extract_and_strip_text(location_element))
    print(extract_and_strip_text(datetime_element))
    print("\n")

Senior Python Developer
Payne, Roberts and Davis
Stewartbury, AA
2021-04-08


Energy engineer
Vasquez-Davidson
Christopherville, AA
2021-04-08


Legal executive
Jackson, Chambers and Levy
Port Ericaburgh, AA
2021-04-08


Fitness centre manager
Savage-Bradley
East Seanview, AP
2021-04-08


Product manager
Ramirez Inc
North Jamieview, AP
2021-04-08


Medical technical officer
Rogers-Yates
Davidville, AP
2021-04-08


Physiological scientist
Kramer-Klein
South Christopher, AE
2021-04-08


Textile designer
Meyers-Johnson
Port Jonathan, AE
2021-04-08


Television floor manager
Hughes-Williams
Osbornetown, AE
2021-04-08


Waste management officer
Jones, Williams and Villa
Scotttown, AP
2021-04-08


Software Engineer (Python)
Garcia PLC
Ericberg, AE
2021-04-08


Interpreter
Gregory and Sons
Ramireztown, AE
2021-04-08


Architect
Clark, Garcia and Sosa
Figueroaview, AA
2021-04-08


Meteorologist
Bush PLC
Kelseystad, AA
2021-04-08


Audiological scientist
Salazar-Meyers
Williamsburgh, AE
2021-04